In [3]:
import numpy as np
import pandas as pd

import time
from pathos.multiprocessing import ProcessingPool as Pool
# from multiprocessing import Pool
from tqdm.notebook import tqdm
import hashlib
import json
import glob
import urllib
import urllib.request as urllib2
import requests
import http
import langid

from goose3 import Goose
from goose3.text import StopWordsChinese, StopWords, StopWordsArabic, StopWordsKorean

import warnings
warnings.filterwarnings('ignore')
print('The script still runs.')


The script still runs.


# gkg data download

In [1]:
import os
import zipfile
import requests
import time
import multiprocessing
from bs4 import BeautifulSoup
from datetime import datetime

path = os.path.split(os.path.abspath('__file__'))[0] + os.sep
data_folder = path + '/Download' + os.sep
base_url = 'http://data.gdeltproject.org/gkg/'

def get_url_list(page_url):
    html = requests.get(page_url).text
    soup = BeautifulSoup(html, 'lxml')
    url_list = []
    for li in soup.body.ul.find_all('li'):
        href = base_url + li.a['href']
        url_list.append(href)
    with open(path + 'url_list.txt', 'w') as f:
        for url in url_list[3:-1]:
            f.write(url + '\n')
    return url_list[3:-1]


def get_url_list_by_span(page_url, begin=20130401, end=20220101):
    url_list_by_span = []
    url_list = get_url_list(page_url)
    for url in url_list:
        filename = url.split('/')[-1]
        date = filename.split('.')[0]
        length = len(str(date))
        if length <= 6:
            if int(str(begin)[:length]) <= int(date) <= int(str(end)[:length]):
                url_list_by_span.append(url)
        else:
            if begin <= int(date) <= end:
                url_list_by_span.append(url)
    with open(path + 'url_list.txt', 'w') as f:
        for url in url_list_by_span:
            f.write(url + '\n')
    return url_list_by_span


def get_url_data(url):
    if not os.path.exists(data_folder):
        os.mkdir(data_folder) 
    filename = url.split('/')[-1]
    filepath = data_folder + filename
    if os.path.exists(filepath) or os.path.exists(filepath[:-4]):
        return
    try:
        data = requests.get(url)
        with open(filepath, "wb") as f:
            f.write(data.content)
        fz = zipfile.ZipFile(filepath, 'r')
        fz.extract(fz.namelist()[0], data_folder)
        fz.close()
        if os.path.exists(filepath):
            os.remove(filepath)
    except Exception as e:
        print(e)
        log = open(path + 'log.txt', 'a')
        log.write(url + '\n')


def main():
    t1 = time.time()
    begin = 20150101
    end = 20240601
    url = base_url + 'index.html'
    url_list = get_url_list_by_span(url, begin, end)
    for i in url_list:
        get_url_data(i)

if __name__ == '__main__':
    main()


# extract and merge GKG data

In [14]:
import os
import zipfile
import pandas as pd
from datetime import datetime
import glob
import numpy as np
import csv

# Define the path to the directory containing the zip files
input_dir = "Download/"
output_dir = "gkg_f1"  # Define your output directory here

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to extract and read CSV files from a zip file
def extract_and_read_csv(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.endswith('.csv'):
                with zip_ref.open(file) as csvfile:
                    df = pd.read_csv(csvfile, on_bad_lines ='skip', encoding = "ISO-8859-1", encoding_errors=None, sep='\t',
                                     quoting=csv.QUOTE_NONE, header=None)
                    df = df[[0, 4, 6, 15, 8, 10, 12, 14, 23]]
                    df.columns = [ 'GKGRECORDID', 'DocumentIdentifier', 'V2Counts', 'V2Tone', 'V2Themes', 'V2Locations', 'V2Persons', 'V2Organizations', 'AllNames']
                    return df
    return None

def merge(g):
    print(g)
    merged_df = pd.DataFrame()
    group_files = glob.glob(input_dir + '\*' + g + '*.gkg.csv.zip')
    for f in group_files:
        df = extract_and_read_csv(f)
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    output_filename = os.path.join(output_dir, f"{g[:4]}-{g[4:6]}-{g[6:8]}")
    merged_df.to_csv(output_filename, index=False, sep='\t')

# import multiprocessing as mp
# pool = mp.Pool()
# res = pool.map(merge, group)

files = glob.glob(input_dir+'\\*.gkg.csv.zip')
group = np.unique([i[8:16] for i in files])
for g in group:
    merge(g)


20230109
20230110
20230111
20230112
20230113
20230114
20230115
20230116
20230117
20230118
20230119
20230120
20230121
20230122
20230123
20230124
20230125
20230126
20230127
20230128
20230129
20230130
20230131
20230201
20230202
20230203
20230204
20230205
20230206
20230207
20230208
20230209
20230210
20230211
20230212
20230213
20230214
20230215
20230216
20230217
20230218
20230219
20230220
20230221
20230222
20230223
20230224
20230225
20230226
20230227
20230228
20230301
20230302
20230303
20230304
20230305
20230306
20230307
20230308
20230309
20230310
20230311
20230312
20230313
20230314
20230315
20230316
20230317
20230318
20230319
20230320
20230321
20230322
20230323
20230324
20230325
20230326
20230327
20230328
20230329
20230330
20230331
20230401
20230402
20230403
20230404
20230405
20230406
20230407
20230408
20230409
20230410
20230411
20230412
20230413
20230414
20230415
20230416
20230417
20230418
20230419
20230420
20230421
20230422
20230423
20230424
20230425
20230426
20230427
20230428
20230429
2

# data screening

In [15]:
mining_theme_list = ['WB_895_MINING_SYSTEMS',
            'WB_1699_METAL_ORE_MINING',
            'WB_566_ENVIRONMENT_AND_NATURAL_RESOURCES',
            'ENV_MINING',
            'WB_1700_NONMETALLIC_MINERAL_MINING_AND_QUARRYING',
            'WB_554_MINING_POLICY_LAWS_AND_REGULATIONS',
            'WB_559_MINING_LAWS_AND_REGULATIONS',
            'WB_2300_EXPLORATORY_DRILLING',
            'WB_1701_MINING_SERVICES',
            'WB_555_ARTISANAL_AND_SMALL_SCALE_MINING',
            'WB_2323_MINING_REGULATION',
            'WB_557_GOVERNANCE_FOR_MINING',
            'WB_2324_MINING_LICENSING_AND_REGISTRATION',
            'MANMADE_DISASTER_MINING_ACCIDENT',
            'MANMADE_DISASTER_MINING_DISASTER',
            'WB_2316_CONFLICT_MINERALS',
            'WB_556_MINING_FISCAL_POLICIES_AND_REVENUE_COLLECTION',
            'WB_2308_MINE_CLOSURE_REGULATION',
            'WB_558_GOVERNMENT_INSTITUTIONS_FOR_MINING',
            'WB_2930_MINING_CADASTRE',
            'WB_897_MINING_ENVIRONMENTAL_MANAGEMENT',
            'WB_2315_STATE_MINING_ENTERPRISES',
            'WB_560_LICENSING_AND_CONTRACTING_OF_MINERALS',
            'WB_2320_MINING_CAPACITY_BUILDING',
            'WB_562_PPP_IN_MINING'
            ]
mining_theme_list = set(mining_theme_list)
len(mining_theme_list)


25

In [16]:
water_theme_list = ['NATURAL_DISASTER_FLASH_FLOOD',
                    'NATURAL_DISASTER_FLASH_FLOODS',
                    'NATURAL_DISASTER_FLOOD',
                    'NATURAL_DISASTER_FLOOD_WARNING',
                    'NATURAL_DISASTER_FLOOD_WATER',
                    'NATURAL_DISASTER_FLOOD_WATERS',
                    'NATURAL_DISASTER_FLOODED',
                    'NATURAL_DISASTER_FLOODED_AREAS',
                    'NATURAL_DISASTER_FLOODING',
                    'NATURAL_DISASTER_FLOODS',
                    'NATURAL_DISASTER_FLOODWATER',
                    'NATURAL_DISASTER_FLOODWATERS',
                    'WB_154_FLOOD_PROTECTION',
                    'NATURAL_DISASTER_HIGH_WATER',
                    'NATURAL_DISASTER_HIGH_WATERS',
                    'NATURAL_DISASTER_WATER_LEVEL',
                    'WATER_SECURITY',
                    'CRISISLEX_C06_WATER_SANITATION',
                    'NATURAL_DISASTER_DROUGHT',
                    'NATURAL_DISASTER_DROUGHTS',
                    'NATURAL_DISASTER_INTENSE_RAINFALL',
                    'NATURAL_DISASTER_TORRENTIAL_RAINFALL',
                    'NATURAL_DISASTER_DESERTIFICATION',
                    'WATER_SECURITY',
                'WB_1000_WATER_MANAGEMENT_STRUCTURES',
                'WB_1002_IRRIGATION_WATER_QUALITY',
                'WB_1021_WATER_LAW',
                'WB_1063_WATER_ALLOCATION_AND_WATER_SUPPLY',
                'WB_1064_WATER_DEMAND_MANAGEMENT',
                'WB_1199_WATER_SUPPLY_AND_SANITATION',
                'WB_1215_WATER_QUALITY_STANDARDS',
                'WB_1220_SURFACE_WATER_MANAGEMENT',
                'WB_137_WATER',
                'WB_138_WATER_SUPPLY',
                'WB_139_SANITATION_AND_WASTEWATER',
                'WB_140_AGRICULTURAL_WATER_MANAGEMENT',
                'WB_141_WATER_RESOURCES_MANAGEMENT',
                'WB_142_ENERGY_AND_WATER',
                'WB_143_RURAL_WATER',
                'WB_144_URBAN_WATER',
                'WB_1462_WATER_SANITATION_AND_HYGIENE',
                'WB_149_WASTEWATER_TREATMENT_AND_DISPOSAL',
                'WB_150_WASTEWATER_REUSE',
                'WB_155_WATERSHED_MANAGEMENT',
                'WB_156_GROUNDWATER_MANAGEMENT',
                'WB_157_ENVIRONMENTAL_WATER_USE_AND_CATCHMENT_PROTECTION',
                'WB_158_WATER_RESOURCES_AND_CLIMATE_ADAPTATION',
                'WB_159_TRANSBOUNDARY_WATER',
                'WB_1729_URBAN_WATER_FINANCIAL_SUSTAINABILITY',
                'WB_1730_URBAN_WATER_COST_RECOVERY',
                'WB_1731_NON_REVENUE_WATER',
                'WB_1778_FRESHWATER_ECOSYSTEMS',
                'WB_1798_WATER_POLLUTION',
                'WB_1994_WATER_FOR_FUEL_EXTRACTION',
                'WB_1998_WATER_ECONOMICS',
                'WB_2005_COMMUNITY_WATER_SUPPLY_MANAGEMENT',
                'WB_2007_WATER_SAFETY_PLANS',
                'WB_2008_WATER_TREATMENT',
                'WB_2009_WATER_QUALITY_MONITORING',
                'WB_2968_URBAN_WATER_AND_SANITATION',
                'WB_2971_WATER_PRICING',
                'WB_2972_GROUNDWATER_CONJUNCTIVE_USE',
                'WB_2978_WATER_POLLUTION_LOAD',
                'WB_2981_DRINKING_WATER_QUALITY_STANDARDS',
                'WB_2982_INDUSTRIAL_WATER_AND_HYGIENE',
                'WB_2984_URBAN_WATER_AND_HYGIENE',
                'WB_2985_RURAL_WATER_AND_HYGIENE',
                'WB_2992_FRESHWATER_FISHERIES',
                'WB_3014_WASTEWATER_DISPOSAL_FACILITIES',
                'WB_423_INTEGRATED_URBAN_WATER_MANAGEMENT',
                'WB_427_WATER_ALLOCATION_AND_WATER_ECONOMICS',
                'WB_991_DRAINAGE_WATER_QUALITY',
                'WB_1710_DROUGHT_RISK_REDUCTION'
                   ]

water_theme_list = set(water_theme_list)
len(water_theme_list)


72

In [ ]:
fs_g = np.unique([f[:14] for f in glob.glob('gkg_f1/20*')])
fs_g = list(reversed(fs_g))
for g in fs_g:
    print(g)
    fs = glob.glob(g + '*')
    data = pd.DataFrame()
    data2 = pd.DataFrame()

    for f in fs:        
        temp = pd.read_csv(f, sep=',')
        if temp.shape[1]<5:
            temp = pd.read_csv(f, sep='\t')
        temp['V2Themes_lst'] = temp['V2Themes'].fillna('').str.split(',')
        temp['V2Themes_lst'] = temp['V2Themes_lst'].map(lambda x: [i.split(';')[-1] for i in x])
        temp['V2Themes_mining_overlap_count'] = temp['V2Themes_lst'].map(lambda x: len(set(x)&mining_theme_list))
        temp = temp[(temp['V2Themes_mining_overlap_count']>2)]
        print(temp.shape)
        data = pd.concat([data, temp], axis=0)

        temp['V2Themes_water_overlap_count'] = temp['V2Themes_lst'].map(lambda x: len(set(x)&water_theme_list))
        temp = temp[(temp['V2Themes_water_overlap_count']>2)]
        data2 = pd.concat([data2, temp], axis=0)

    data = data.drop(['V2Themes_lst'], axis=1)
    print('Mining data shape: ', data.shape[0])
    data.to_csv(g.replace('f1', 'f1_mining') + '.csv', index=None)    

    data2 = data2.drop(['V2Themes_lst'], axis=1)
    print('Mining water data shape: ', data2.shape[0])
    data2.to_csv(g.replace('f1', 'f1_mining_water') + '.csv', index=None)    

